# NERSC Cluster Deploy Tutorial: Tuning Hyperparameters of a Distributed PyTorch Model with PBT using Ray Train & Tune

📖 [Back to Table of Contents](../README.md)<br>
<!-- ⬅ [Previous notebook](./ex_01_pytorch_ray_hvd.ipynb) <br> -->
➡ [Next notebook](./ex_02_tensorflow_ray_train_tune.ipynb) <br>

----


## Introduction

We are going to run an example Ray Train & Tune code. This example looks at tunning hyperparameters of a distrbuted PyTorch Model with PBT. This tutorial is following the code in this example: https://docs.ray.io/en/latest/train/examples/pytorch/tune_cifar_torch_pbt_example.html

> **Note**:
> To setup the environment for the notebook, execute on command line: `./setup.sh 1` then select the kernel `pytorch-1.13.1` in the notebook

This Ray cluster will be setup using the NERSC PyTorch module and deployed on Perlmutter.



# Starting Ray Cluster

## Superfacility API

To deploy the Ray cluster via the NERSC Superfacility API you require a valid API client. 

To create a valid client visit your profile page in [Iris](https://iris.nersc.gov/):

<img src="img/iris_profile_header.png" width="800" />

Then scroll down to the **Superfacility API Clients** section and click the "+ New Client" button which will produce this window:

<img src="img/new_sf_api_client.png" width="400" />

To submit and deploy a Ray cluster we require the highest security level (<span style="color:red">RED</span>). **[This client id is valid for 2 days]**

Once created then saved the `client_id` string and `private_key` dictionary (you can also save the private key in PEM format) ready for use with the `SuperfacilityAPI` library.

> **Note**:
> This step should only be repeated if your client has expired


For more information about the NERSC Superfacility API visit the [documenation](https://docs.nersc.gov/services/sfapi/).

In [4]:
from SuperfacilityAPI import SuperfacilityAPI, SuperfacilityAccessToken
from utility import load_secrets

# Replace with your client id string and private key dictionary
client_id, private_key = load_secrets()
# client_id = "<your client id string>"
# private_key = "<your private key dict>"

api_key = SuperfacilityAccessToken(
    client_id = client_id,
    private_key = private_key
)
sfp_api = SuperfacilityAPI(api_key)

## Creating Ray Cluster

To create a ray cluster on NERSC compute nodes, execute the `deploy_ray_cluster` function with your desired slurm sbatch options.

In [5]:
from nersc_cluster_deploy import deploy_ray_cluster
from utility import user_account

slurm_options = {
    'qos': 'debug',
    'account': user_account(),
    'nodes': '2',
    't': '00:30:00'
}
site = 'perlmutter'
module_load = 'pytorch/1.13.1'

job = deploy_ray_cluster(
    sfp_api,
    slurm_options,
    site,
    job_setup = [f'module load {module_load}']
)

In [6]:
job

{'error': None, 'jobid': '5906466', 'task_id': '11931'}

Now the job has been submitted, check on the job status

In [28]:
import os
import pandas as pd
sqs_table = sfp_api.get_jobs(site=site, user=os.getlogin(), sacct=False)
sqs_df = pd.DataFrame(sqs_table['output'])
sqs_df

,account,tres_per_node,min_cpus,min_tmp_disk,end_time,features,group,over_subscribe,jobid,name,...,partition,nodelist(reason),start_time,state,uid,submit_time,licenses,core_spec,schednodes,work_dir
0,dasrepo_g,N/A,128,0,2023-03-03T19:45:32,gpu&a100&hbm40g,75235,NO,5906466,sbatch,...,gpu_ss11,nid[003044-003045],2023-03-03T19:15:32,RUNNING,75235,2023-03-03T19:15:32,u2:1,N/A,(null),/global/u2/a/asnaylor


Check job log

In [27]:
!cat ~/slurm-{job['jobid']}.out

In case of issues, please refer to our known issues: https://docs.nersc.gov/current/
and open a help ticket if your issue is not listed: https://help.nersc.gov/
[slurm] - Starting ray HEAD
2023-03-03 19:15:38,460	INFO usage_lib.py:435 -- Usage stats collection is disabled.
2023-03-03 19:15:38,460	INFO scripts.py:710 -- Local node IP: nid003044
2023-03-03 19:15:40,974	SUCC scripts.py:747 -- --------------------
2023-03-03 19:15:40,974	SUCC scripts.py:748 -- Ray runtime started.
2023-03-03 19:15:40,974	SUCC scripts.py:749 -- --------------------
2023-03-03 19:15:40,974	INFO scripts.py:751 -- Next steps
2023-03-03 19:15:40,974	INFO scripts.py:752 -- To connect to this Ray runtime from another node, run
2023-03-03 19:15:40,974	INFO scripts.py:755 --   ray start --address='nid003044:6379'
2023-03-03 19:15:40,974	INFO scripts.py:771 -- Alternatively, use the following Python code:
2023-03-03 19:15:40,974	INFO scripts.py:773 -- import ray
2023-03-03 19:15:40,974	INFO scripts.py:777 -- ray.ini

## Connect to Ray Cluster

Get the Ray cluster head node ip address to connect to the cluster

In [29]:
from nersc_cluster_deploy import get_ray_cluster_address
import ray

cluster_address = get_ray_cluster_address(
    sfp_api,
    job['jobid'],
    site
)
ray.init(cluster_address)

Python version:,3.9.15
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


Check all nodes connected to cluster

In [30]:
from nersc_cluster_deploy import ray_cluster_summary

ray_cluster_summary()

Cluster Summary
---------------
Nodes: 2
CPU:   256
GPU:   8
RAM:   309.42 GB


## Setup PyTorch Model

In [31]:
import argparse
import os

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from filelock import FileLock
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18

import ray
import ray.train as train
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.air.config import FailureConfig, RunConfig, ScalingConfig
from ray.train.torch import TorchTrainer
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.tune_config import TuneConfig
from ray.tune.tuner import Tuner

In [32]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) // session.get_world_size()
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def validate_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) // session.get_world_size()
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n "
        f"Accuracy: {(100 * correct):>0.1f}%, "
        f"Avg loss: {test_loss:>8f} \n"
    )
    return {"loss": test_loss}


def update_optimizer_config(optimizer, config):
    for param_group in optimizer.param_groups:
        for param, val in config.items():
            param_group[param] = val


def train_func(config):
    epochs = config.get("epochs", 3)

    model = resnet18()

    # Note that `prepare_model` needs to be called before setting optimizer.
    if not session.get_checkpoint():  # fresh start
        model = train.torch.prepare_model(model)

    # Create optimizer.
    optimizer_config = {
        "lr": config.get("lr"),
        "momentum": config.get("momentum"),
    }
    optimizer = torch.optim.SGD(model.parameters(), **optimizer_config)

    starting_epoch = 0
    if session.get_checkpoint():
        checkpoint_dict = session.get_checkpoint().to_dict()

        # Load in model
        model_state = checkpoint_dict["model"]
        model.load_state_dict(model_state)
        model = train.torch.prepare_model(model)

        # Load in optimizer
        optimizer_state = checkpoint_dict["optimizer_state_dict"]
        optimizer.load_state_dict(optimizer_state)

        # Optimizer configs (`lr`, `momentum`) are being mutated by PBT and passed in
        # through config, so we need to update the optimizer loaded from the checkpoint
        update_optimizer_config(optimizer, optimizer_config)

        # The current epoch increments the loaded epoch by 1
        checkpoint_epoch = checkpoint_dict["epoch"]
        starting_epoch = checkpoint_epoch + 1

    # Load in training and validation data.
    transform_train = transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )  # meanstd transformation

    transform_test = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )

    data_dir = config.get("data_dir", os.path.expanduser("~/data"))
    os.makedirs(data_dir, exist_ok=True)
    with FileLock(os.path.join(data_dir, ".ray.lock")):
        train_dataset = CIFAR10(
            root=data_dir, train=True, download=True, transform=transform_train
        )
        validation_dataset = CIFAR10(
            root=data_dir, train=False, download=False, transform=transform_test
        )

    if config.get("test_mode"):
        train_dataset = Subset(train_dataset, list(range(64)))
        validation_dataset = Subset(validation_dataset, list(range(64)))

    worker_batch_size = config["batch_size"] // session.get_world_size()

    train_loader = DataLoader(train_dataset, batch_size=worker_batch_size)
    validation_loader = DataLoader(validation_dataset, batch_size=worker_batch_size)

    train_loader = train.torch.prepare_data_loader(train_loader)
    validation_loader = train.torch.prepare_data_loader(validation_loader)

    # Create loss.
    criterion = nn.CrossEntropyLoss()

    for epoch in range(starting_epoch, epochs):
        train_epoch(train_loader, model, criterion, optimizer)
        result = validate_epoch(validation_loader, model, criterion)
        checkpoint = Checkpoint.from_dict(
            {
                "epoch": epoch,
                "model": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            }
        )

        session.report(result, checkpoint=checkpoint)


## Train Model

In [33]:
SCRATCH = os.getenv('SCRATCH')

node_resources = ray.cluster_resources()
num_workers = int(node_resources['GPU'])
use_gpu = True

data_dir = os.path.join(SCRATCH, 'CIFAR10')
num_epochs = 5
smoke_test = False
synch = False

In [34]:
trainer = TorchTrainer(
        train_func,
        scaling_config=ScalingConfig(
            num_workers=num_workers, use_gpu=use_gpu
        ),
    )

In [35]:
pbt_scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=1,
        hyperparam_mutations={
            "train_loop_config": {
                # distribution for resampling
                "lr": tune.loguniform(0.001, 0.1),
                # allow perturbations within this set of categorical values
                "momentum": [0.8, 0.9, 0.99],
            }
        },
        synch=synch,
    )

In [36]:
tuner = Tuner(
        trainer,
        param_space={
            "train_loop_config": {
                "lr": tune.grid_search([0.001, 0.01, 0.05, 0.1]),
                "momentum": 0.8,
                "batch_size": 128 * num_workers,
                "test_mode": smoke_test,  # whether to to subset the data
                "data_dir": data_dir,
                "epochs": num_epochs,
            }
        },
        tune_config=TuneConfig(
            num_samples=1, metric="loss", mode="min", scheduler=pbt_scheduler
        ),
        run_config=RunConfig(
            stop={"training_iteration": 3 if smoke_test else num_epochs},
            failure_config=FailureConfig(max_failures=3),  # used for fault tolerance
        ),
    )


In [37]:
results = tuner.fit()

(TunerInternal pid=61223) /global/homes/a/asnaylor/.local/perlmutter/pytorch1.13.1/lib/python3.9/site-packages/ray/tune/tune.py:562: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
(TunerInternal pid=61223)   warnings.warn(
(TunerInternal pid=61223) 2023-03-03 19:20:48,989	WARNING trial_runner.py:369 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (281 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
(TunerInternal pid=61223) 2023-03-03 19:20:48,993	WARNING trial_runner.py:1677 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for you

(RayTrainWorker pid=61784) Files already downloaded and verified
(RayTrainWorker pid=61787) Files already downloaded and verified
(RayTrainWorker pid=61786) Files already downloaded and verified
(RayTrainWorker pid=61785) Files already downloaded and verified
(RayTrainWorker pid=98657, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=98658, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=98660, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=98659, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=98659, ip=128.55.69.178) loss: 7.233888  [    0/ 6250]
(RayTrainWorker pid=98658, ip=128.55.69.178) loss: 7.088008  [    0/ 6250]
(RayTrainWorker pid=98657, ip=128.55.69.178) loss: 7.143627  [    0/ 6250]
(RayTrainWorker pid=98660, ip=128.55.69.178) loss: 7.204629  [    0/ 6250]
(RayTrainWorker pid=61784) loss: 7.228695  [    0/ 6250]
(RayTrainWorker pid=61787) loss: 7.124676  [    0/ 

(RayTrainWorker pid=62849) 2023-03-03 19:21:23,577	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=8]
(RayTrainWorker pid=62849) 2023-03-03 19:21:25,488	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=99128, ip=128.55.69.178) 2023-03-03 19:21:25,529	INFO train_loop_utils.py:255 -- Moving model to device: cuda:1
(RayTrainWorker pid=99128, ip=128.55.69.178) 2023-03-03 19:21:27,023	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=62849) 2023-03-03 19:21:27,085	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=62850) Files already downloaded and verified
(RayTrainWorker pid=62849) Files already downloaded and verified
(RayTrainWorker pid=62852) Files already downloaded and verified
(RayTrainWorker pid=99131, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=99130, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=99129, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=99128, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=62851) Files already downloaded and verified
(RayTrainWorker pid=62852) loss: 7.217724  [    0/ 6250]
(RayTrainWorker pid=62851) loss: 7.198362  [    0/ 6250]
(RayTrainWorker pid=62850) loss: 7.171299  [    0/ 6250]
(RayTrainWorker pid=62849) loss: 7.220249  [    0/ 6250]
(RayTrainWorker pid=99131, ip=128.55.69.178) loss: 7.113987  [    0/ 6250]
(RayTrainWorker pid=99130, ip=128.55.69.178) loss: 6.981210  [    0/ 6250]
(RayTrainWorker pid=99129, ip=

(RayTrainWorker pid=63837) 2023-03-03 19:21:45,564	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=8]
(RayTrainWorker pid=63837) 2023-03-03 19:21:47,329	INFO train_loop_utils.py:255 -- Moving model to device: cuda:0
(RayTrainWorker pid=99625, ip=128.55.69.178) 2023-03-03 19:21:47,395	INFO train_loop_utils.py:255 -- Moving model to device: cuda:0
(RayTrainWorker pid=99625, ip=128.55.69.178) 2023-03-03 19:21:48,908	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=63837) 2023-03-03 19:21:49,001	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.


...

(TunerInternal pid=61223) 2023-03-03 19:28:23,260	WARNING util.py:244 -- The `process_trial_save` operation took 0.967 s, which may be a performance bottleneck.
(TorchTrainer pid=80619) 2023-03-03 19:28:27,252	INFO trainable.py:791 -- Restored on nid003044 from checkpoint: /global/homes/a/asnaylor/ray_results/TorchTrainer_2023-03-03_19-20-47/TorchTrainer_8f5ae_00002_2_lr=0.0500_2023-03-03_19-21-40/checkpoint_tmpf40f8a
(TorchTrainer pid=80619) 2023-03-03 19:28:27,252	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': None, '_time_total': 79.00595831871033, '_episodes_total': None}
(RayTrainWorker pid=80753) 2023-03-03 19:28:29,786	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=8]
(RayTrainWorker pid=80753) 2023-03-03 19:28:33,048	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=109245, ip=128.55.69.178) 2023-03-03 19:28:33,058	INFO train_loop_utils.py:255 -- Moving model to device

(RayTrainWorker pid=109245, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=109247, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=80756) Files already downloaded and verified
(RayTrainWorker pid=80754) Files already downloaded and verified
(RayTrainWorker pid=80753) Files already downloaded and verified
(RayTrainWorker pid=80755) Files already downloaded and verified
(RayTrainWorker pid=109248, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=109246, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=109245, ip=128.55.69.178) loss: 1.069553  [    0/ 6250]
(RayTrainWorker pid=109248, ip=128.55.69.178) loss: 1.230193  [    0/ 6250]
(RayTrainWorker pid=109247, ip=128.55.69.178) loss: 1.148094  [    0/ 6250]
(RayTrainWorker pid=109246, ip=128.55.69.178) loss: 1.196400  [    0/ 6250]
(RayTrainWorker pid=80756) loss: 1.185359  [    0/ 6250]
(RayTrainWorker pid=80755) loss: 1.112800  

(TorchTrainer pid=81595) 2023-03-03 19:28:50,685	INFO trainable.py:791 -- Restored on nid003044 from checkpoint: /global/homes/a/asnaylor/ray_results/TorchTrainer_2023-03-03_19-20-47/TorchTrainer_8f5ae_00003_3_lr=0.1000_2023-03-03_19-22-02/checkpoint_tmpba0165
(TorchTrainer pid=81595) 2023-03-03 19:28:50,686	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': None, '_time_total': 79.01619005203247, '_episodes_total': None}
(RayTrainWorker pid=81741) 2023-03-03 19:28:53,614	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=8]
(RayTrainWorker pid=81741) 2023-03-03 19:28:57,087	INFO train_loop_utils.py:255 -- Moving model to device: cuda:1
(RayTrainWorker pid=109753, ip=128.55.69.178) 2023-03-03 19:28:57,067	INFO train_loop_utils.py:255 -- Moving model to device: cuda:1
(RayTrainWorker pid=109753, ip=128.55.69.178) 2023-03-03 19:28:58,582	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel

(RayTrainWorker pid=109754, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=109753, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=109755, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=81743) Files already downloaded and verified
(RayTrainWorker pid=81741) Files already downloaded and verified
(RayTrainWorker pid=81742) Files already downloaded and verified
(RayTrainWorker pid=81744) Files already downloaded and verified
(RayTrainWorker pid=109756, ip=128.55.69.178) Files already downloaded and verified
(RayTrainWorker pid=109754, ip=128.55.69.178) loss: 1.128427  [    0/ 6250]
(RayTrainWorker pid=109753, ip=128.55.69.178) loss: 1.078292  [    0/ 6250]
(RayTrainWorker pid=109755, ip=128.55.69.178) loss: 1.117896  [    0/ 6250]
(RayTrainWorker pid=109756, ip=128.55.69.178) loss: 1.147443  [    0/ 6250]
(RayTrainWorker pid=81741) loss: 1.113449  [    0/ 6250]
(RayTrainWorker pid=81744) loss: 1.137184  

(TunerInternal pid=61223) 2023-03-03 19:29:11,126	INFO tune.py:798 -- Total run time: 502.20 seconds (502.13 seconds for the tuning loop).


In [38]:
print(results.get_best_result(metric="loss", mode="min"))

Result(metrics={'loss': 0.9854333639144898, '_timestamp': 1677900479, '_time_this_iter_s': 15.428494215011597, '_training_iteration': 1, 'should_checkpoint': True, 'done': True, 'trial_id': '8f5ae_00000', 'experiment_tag': '0_lr=0.0010@perturbed[train_loop_config=lr_0_06_momentum_0_9_batch_size_1024_test_mode_False_data_dir_pscratch_sd_a_asnaylor_CIFAR10_epochs_5]'}, error=None, log_dir=PosixPath('/global/homes/a/asnaylor/ray_results/TorchTrainer_2023-03-03_19-20-47/TorchTrainer_8f5ae_00000_0_lr=0.0010_2023-03-03_19-20-49'))


## Close cluster conection and stop job

In [39]:
ray.shutdown()

In [40]:
sfp_api.delete_job(site, job['jobid'])

{'task_id': '0', 'status': 'OK', 'error': None}

## Explore Training in Tensorboard

In [39]:
import nersc_tensorboard_helper
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [40]:
log_dir = str(results.get_best_result(metric="loss", mode="min").log_dir)

In [41]:
%tensorboard --logdir $log_dir --port 0

In [42]:
nersc_tensorboard_helper.tb_address()